In [ ]:
!pip install bardapi
!pip install git+https://github.com/openai/whisper.git
!pip install python-docx
#!pip install git+https://github.com/dsdanielpark/Bard-API.git
!sudo apt update && sudo apt install ffmpeg
!pip install bert-extractive-summarizer


In [ ]:
#DO NOT TOUCH
import tkinter as tk
from tkinter import filedialog
from summarizer import Summarizer
from docx import Document
from bardapi import Bard
import os
import threading
import time
from PIL import Image, ImageTk

# Initialize the Whisper model
import importlib

# Import the Whisper module dynamically
whisper = importlib.import_module("whisper")
# Load the Whisper model
model = whisper.load_model("base")

# Initialize global variables
audio_file_path = ""
processing_finished = False
result_text = ""
gif_index = 0  # Define gif_index as a global variable

def load_audio():
    global audio_file_path
    audio_file_path = filedialog.askopenfilename(filetypes=[("Audio files", "*.mp3")])
    if audio_file_path:
        loading_label.config(text="Loading...")
        processing_thread = threading.Thread(target=process_audio)
        processing_thread.start()
        input_frame.pack_forget()
        loading_frame.pack()

def process_audio():
    global processing_finished, result_text
    result_text = "Processing failed. Please check your audio file and try again."

    if audio_file_path:
        try:
            # Transcribe the audio file
            result = model.transcribe(audio_file_path)
            transcription = result["text"]

            # Save the transcription as a text file
            with open("transcription.txt", "w", encoding="utf-8") as txt:
                txt.write(transcription)

            # Read the text from the file
            with open('transcription.txt', 'r', encoding='utf-8') as file:
                text = file.read()

            # Initialize the summarizer
            summarizer = Summarizer()

            # Summarize the text
            summary = summarizer(text)

            # Save the summarized text in a text file
            with open("summary.txt", "w", encoding="utf-8") as summary_txt:
                summary_txt.write(summary)

            # Initialize Bard
            os.environ['_BARD_API_KEY'] = "cghhfj9P3KnUCOP6kXhGvFN5l1-ugSqZjM_hheeEKbdsjPHM7L_Pi2x03a04ReybOqr6bw."
            prompt_topics = f"Please analyze the given study notes and provide a list of all the topics covered, without any additional information.\n{summary}Please don't write any conclusion. Do not write"

            # Use Bard to get the topics for the entire text
            response_topics = get_completion(prompt_topics)

            # Create a Word document
            doc = Document()
            doc.add_heading('Topics Covered', level=1)
            doc.add_paragraph(response_topics)

            # Split the text into sentences
            sentences = text.split('.')

            # Split the sentences into parts with 100 sentences each
            parts = [sentences[i:i + 100] for i in range(0, len(sentences), 100)]

            # Initialize an empty list to store the generated summaries
            summaries = []

            # Loop through the text parts and generate summaries
            for part in parts:
                prompt_notes = f"""A Convert the attached lecture notes into comprehensive study notes, suitable for college students.
Ensure that the content remains faithful to the original lecture notes, without introducing any additional information.
Condense and rephrase the provided lecture notes while retaining all essential concepts and details.
Organize the study notes in a logical way, using headings, subheadings, and bullet points as appropriate.
The resulting study notes should be clear, concise, and easy to understand. Avoid adding your own conclusions or recommendations beyond what is covered in the lecture notes.
Below are my lecture notes:
{part}
"""
                response_notes = get_completion(prompt_notes)
                summaries.append(response_notes)
                doc.add_heading('Summary', level=2)
                doc.add_paragraph(response_notes)
                time.sleep(19)

            # Save the Word document with topics and summaries
            doc.save('.//bard_output.docx')

            # Load the Word document
            doc = Document('.//bard_output.docx')
            for paragraph in doc.paragraphs:
                if "**" in paragraph.text:
                    paragraph.text = paragraph.text.replace("**", "")

            for paragraph in doc.paragraphs:
                if '*' in paragraph.text:
                    paragraph.text = paragraph.text.replace('*', '•')

            doc.save('.//bard_output2.docx')

            result_text = response_notes  # Display the summary in the result text

        except Exception as e:
            print(e)
            result_text = "Processing failed. An error occurred."

    processing_finished = True
    # Schedule UI update on the main thread
    app.after(1, update_ui)

def get_completion(prompt):
    response = Bard().get_answer(prompt)['content']
    return response

def download_result():
    result_file_path = './/bard_output2.docx'  # Update with your file path
    file_to_download = open(result_file_path, 'rb').read()
    download_dialog = filedialog.asksaveasfilename(defaultextension=".docx", filetypes=[("Word files", "*.docx")])
    if download_dialog:
        with open(download_dialog, 'wb') as f:
            f.write(file_to_download)

def update_ui():
    if processing_finished:
        loading_frame.pack_forget()  # Hide the loading frame
        result_label.config(text="Your customized notes are ready :)", fg="black")  # Display the label with text color
        result_text_box.config(state=tk.NORMAL)
        result_text_box.delete(1.0, tk.END)  # Clear the text box
        result_text_box.insert(tk.END, result_text)  # Update the text box
        result_text_box.config(state=tk.DISABLED)  # Disable text box editing
        download_button.config(state=tk.NORMAL)
        result_frame.pack()  # Show the result frame
    else:
        loading_label.config(text="Loading...")

# Create the tkinter application
app = tk.Tk()
app.title("Lets Note")
app.iconbitmap("\\logo.ico")

# Set the size of the main window (adjust width and height as needed)
app.geometry("800x600")

# Set background color for the application
app.configure(background="#FBB9BC")

# Set background color for frames
bg_color = "#FBB9BC"

# First Page: Input Audio File
input_frame = tk.Frame(app, bg=bg_color)
input_frame.pack()

input_label = tk.Label(input_frame, text="", font=("Erostulla", 44), bg=bg_color)
input_label.grid(row=0, column=0, columnspan=2)  # Place the label in row 0, column 0, spanning 2 columns

input_label = tk.Label(input_frame, text="Lets Notes!", font=("Gerallydine", 44), bg=bg_color)
input_label.grid(row=4, column=0, columnspan=2)  # Place the label in row 0, column 0, spanning 2 columns

load_button = tk.Button(input_frame, text="Choose Lecture", command=load_audio, bg="#B0C6D2", font=("Mongolian Baiti", 14))
load_button.grid(row=8, column=0, columnspan=2, pady=10, padx=10)  # Place the button in row 8, column 0, spanning 2 columns and adding padding

# Second Page: Loading
loading_frame = tk.Frame(app, bg=bg_color)
loading_label = tk.Label(loading_frame, text="Loading...", font=("Mongolian Baiti", 20), bg=bg_color)

loading_label.pack()

gif_frames = []
gif_path = "riyal.gif"  # Change this to the path of your GIF
gif = Image.open(gif_path)
gif_width, gif_height = gif.size
for frame in range(gif.n_frames):
    gif.seek(frame)
    gif_frame = gif.copy().resize((gif_width, gif_height))
    gif_frames.append(ImageTk.PhotoImage(gif_frame))

# Create a canvas within the loading frame to display the GIF
canvas_loading = tk.Canvas(loading_frame, width=gif.width, height=gif.height, bg="#FBB9BC")
canvas_loading.pack()  # Pack the canvas to ensure it's displayed

def update_gif():
    global gif_index
    canvas_loading.create_image(0, 0, anchor=tk.NW, image=gif_frames[gif_index])
    gif_index = (gif_index + 1) % len(gif_frames)
    app.after(100, update_gif)  # Update the GIF every 100 milliseconds

update_gif()  # Start updating the GIF

# Third Page: Display Result
result_frame = tk.Frame(app, bg=bg_color)
result_label = tk.Label(result_frame, text="", font=("Mongolian Baiti", 20), bg="#FCCDA1")
result_label.grid(row=0, column=0, columnspan=2, pady=20)

# Create a Canvas for custom font color
canvas_result_label = tk.Canvas(result_label, width=200, height=30, bg="#FCCDA1", highlightthickness=0)
canvas_result_label.create_text(100, 15, text="Your customized notes are ready :)", fill="black", font=("Mongolian Baiti", 20))

result_text_box = tk.Text(result_frame, wrap=tk.WORD, state=tk.DISABLED, height=30, width=60)
result_text_box.grid(row=1, column=0, columnspan=2, sticky="nsew")

scrollbar = tk.Scrollbar(result_frame, command=result_text_box.yview, orient=tk.VERTICAL)
scrollbar.grid(row=1, column=2, sticky="ns")

result_text_box.config(yscrollcommand=scrollbar.set)

download_button = tk.Button(result_frame, text="Download", command=download_result, bg="#B0C6D2", font=("Mongolian Baiti", 14))
download_button.grid(row=8, column=0, columnspan=2, pady=10, padx=10)

# Start the tkinter main event loop
app.mainloop()
